In [10]:
import torch
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('hfl/Chinese-bert-wwm')
model = BertModel.from_pretrained('hfl/Chinese-bert-wwm')

def sentence_to_embedding(sentence):
    tokens = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True, max_length=128)
    input_ids = tokens['input_ids']
    attention_mask = tokens['attention_mask']
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    sentence_embedding = outputs.last_hidden_state[:, 0, :]
    return sentence_embedding

d:\Anaconda\envs\gpaTorch\lib\site-packages\transformers\utils\generic.py:482: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
d:\Anaconda\envs\gpaTorch\lib\site-packages\transformers\utils\generic.py:339: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [11]:
import numpy as np
import pandas as pd

# 读取jsonl文件
file_path = 'comment_20240626225604.jsonl'
df = pd.read_json(file_path, lines=True)

# 提取content字段
contents = df['content'].tolist()

In [12]:
import re

for i in range(len(contents)):
    if contents[i].startswith("回复@") or contents[i].startswith("回复 @"):
        pattern_reply = r'回复.*?:'
        contents[i] = re.sub(pattern_reply, '', contents[i])
    pattern_urls = r'http[s]?://\S+'
    contents[i] = re.sub(pattern_urls, '', contents[i])
    pattern = r'#.*?#'
    contents[i] = re.sub(pattern, '', contents[i])

In [13]:
embeddings = [sentence_to_embedding(sentence) for sentence in contents]

In [14]:
using_list = []
for i in range(len(embeddings)):
    using_list.append(embeddings[i].squeeze().numpy())

In [15]:
numpy_list = np.array(using_list)

In [16]:
np.save('numpy_list.npy', numpy_list)

In [17]:
loaded_array = np.load('numpy_list.npy')

In [18]:
# K-means
 
class MyKmeans:
    # k是K-means的指定的聚类个数，n是迭代次数
    def __init__(self, k, n=20):
        self.k = k
        self.n = n
        
    def fit(self, x, centers=None):
        # 随机选择k个点, 或者指定k个点
        if centers is None:
            idx = np.random.randint(low=0, high=len(x), size=self.k)
            centers = x[idx]
        
        inters = 0
        while inters < self.n:
            # 初始化字典，存储每个聚类中心对应的点集
            points_set = {key: [] for key in range(self.k)}
 
            for idx, p in enumerate(x):
                nearest_index = np.argmin(np.sum((centers - p) ** 2, axis=1))
                points_set[nearest_index].append((p, idx))
 
            for i_k in range(self.k):
                if len(points_set[i_k]) > 0:
                    centers[i_k] = sum(p for p, _ in points_set[i_k]) / len(points_set[i_k])
                
            inters += 1
 
        return points_set, centers

In [19]:
m_result = MyKmeans(3)
points_set, centers = m_result.fit(loaded_array)

In [20]:
for cluster_idx, points in points_set.items():
    print(f"Cluster {cluster_idx}:")
    print(len(points))
    for point, idx in points:
        print(f"  Sentence {idx}: {contents[idx]}")

Cluster 0:
2164
  Sentence 2: 我也是单身家庭长大，从小没有父母的陪伴，心里极度缺爱，博主说的真到心坎去了，一样一样的～[抱抱]我懂，我也真的能理解[抱抱]所以我赞同博主说的
  Sentence 3: 这个胖猫，挺牛，国服梦奇。游戏天才 ，2，3年挣5，60万。天妒英才啊。
  Sentence 13:   从小卖蒸馍啥事都经过，从小打游戏🎮呢？先要爱自己。
  Sentence 17: 我想说附近的流浪汉这几天饿不着了。
  Sentence 19: 我和这个女的同龄，很难理解，都这么大年纪了，怎么好意思花小男孩钱的。各种PUA，到最后说自己是女同。这女的戏怎么这么多。
  Sentence 26: 知道吗？现实中啥都不图的女人也输得好惨，也没有被珍惜，这个社会不知道怎么了
  Sentence 27: 别的女生想找个好男人男的很，她轻易践踏一个这么善良单纯一心为她的男生生命，真不怪当今这个世界人与人之间信任崩塌
  Sentence 33: 吃了一晚上瓜，只想说，为什么这种男生我遇不到啊！！！我真的是那种，你对我好，我会加倍还你的那种……… 另外真的想说，珍爱生命，远离npd人格！！！
  Sentence 37: 在爱情里单方面付出并为之付出性命的都是傻子，太可惜了唉
  Sentence 38: 我都不知道朝哪个方向磕头能求来的理想型被人当狗玩🙃这个世界是真的癫
  Sentence 39: 我曾经跟很多兄弟朋友说，不要和女拳去浪费口水辩，因为她们的立场就是无条件站在女性一方，不论是非对错，哪怕你是劳荣枝翟欣欣马蓉小慧君，都依然会为其开脱，所以何必跟这种人浪费时间，只要正确记录历史，剩下的让大众去选择相信和评判就够了。
  Sentence 40: 我再举个例，路人们仔细看图8，哪怕你是女性，她们依然会攻击你，还是会避重就轻，无法就事论事，哪怕大家都在看似讨论这件事，她们会用一切的攻击方法排除异己，不加入那就攻击谩骂，用一切难听的词汇问候你，这就是女拳。
  Sentence 41: 我只能说，只有女性的世界该有多美好啊
  Sentence 45: 我不明白恋爱都是你情我愿的事情，不是说男女平等吗。那有的女的大着肚子跳楼的，生完孩子自杀的，咋没看见网上这么网暴一个男人。女方有错，可以用法律制裁。自己私下解决呗。博取别人同情，网